In [1]:
import numpy as np
import pandas as pd
import os

model_file = pd.read_excel('/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/model_file.xlsx')

model_file

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [2]:
model_file.loc[model_file['Emotion'] == '공포', 'Emotion'] = '불안'

In [3]:
model_file.isna().sum()

Sentence    0
Emotion     0
dtype: int64

In [4]:
model_file.loc[:5]

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,불안
1,그냥 내 느낌일뿐겠지?,불안
2,아직너무초기라서 그런거죠?,불안
3,유치원버스 사고 낫다던데,불안
4,근데 원래이런거맞나요,불안
5,남자친구가 떠날까봐요,불안


In [5]:
model_file[model_file['Emotion'] == '놀람']

,Sentence,Emotion
5468,7로 나온다든데 아니었나,놀람
5469,의사양반 내가고자라니!,놀람
5470,이 사람 얼굴이 약간 전라도 사람처럼 생겼네,놀람
5471,아직도 안잡았냐??,놀람
5472,이분 ~~진짜 욕먹는건 이제 신의 영역에 들어선거같다~~ㅋㅋ,놀람
...,...,...
11361,시대가 어느시대인데 사람이 뽑냐,놀람
11362,진심 내 인생 드라마임..,놀람
11363,대단하다... 진짜,놀람
11364,헐 김종민이가 대상급인가?,놀람


In [6]:
model_file = model_file.drop(model_file[model_file['Emotion'] == '놀람'].index)

In [7]:
model_file[model_file['Emotion'] == '놀람']

,Sentence,Emotion


## 불안 = 0, 분노 = 1, 슬픔 = 2, 중립 = 3, 행복 = 4, 혐오 = 5

In [8]:
Emotions = ['불안', '분노', '슬픔', '중립', '행복', '혐오']

for label, emotion in enumerate(Emotions):
    model_file.loc[model_file['Emotion'] == emotion, 'Emotion'] = label

In [9]:
model_file = model_file.reset_index()

In [10]:
model_file

,index,Sentence,Emotion
0,0,언니 동생으로 부르는게 맞는 일인가요..??,0
1,1,그냥 내 느낌일뿐겠지?,0
2,2,아직너무초기라서 그런거죠?,0
3,3,유치원버스 사고 낫다던데,0
4,4,근데 원래이런거맞나요,0
...,...,...,...
32691,38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,5
32692,38590,재미가 없으니 망하지,5
32693,38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,5
32694,38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,5


In [11]:
!pip install konlpy

In [12]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-06-14 22:39:42.950322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
x = model_file['Sentence']
y = model_file['Emotion']
x

0                                 언니 동생으로 부르는게 맞는 일인가요..??
1                                             그냥 내 느낌일뿐겠지?
2                                           아직너무초기라서 그런거죠?
3                                            유치원버스 사고 낫다던데
4                                              근데 원래이런거맞나요
                               ...                        
32691                 솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..
32692                                          재미가 없으니 망하지
32693    공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...
32694                 코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ
32695                 와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요
Name: Sentence, Length: 32696, dtype: object

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

In [16]:
len(x_train), len(y_train), len(x_test), len(y_test)

(26156, 26156, 6540, 6540)

In [17]:
#전처리 함수 만들기
def preprocessing(text, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  texts = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',text)
  
  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(texts, stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [18]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_text = []

for text in x_train:
  # 텍스트가 문자열인 경우만 전처리 진행
  if type(text) == str:
    clean_train_text.append(preprocessing(text, okt, remove_stopwords=True,stop_words= stop_words))
  else:
    clean_train_text.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_text

[['결국', '나라', '를', '망하다', '몰다'],
 ['서민',
  '적다',
  '액수',
  '에도',
  '내',
  '물죄',
  '성립',
  '데',
  '돼다',
  '웃',
  '분',
  '얼',
  '마을',
  '먹다',
  '내',
  '물죄',
  '성립',
  '되다'],
 ['이제', '학기', '남다', '어떻다', '하다', '되다', '생각', '들다', '자신', '없어지다'],
 ['저', '같이', '회계', '에', '무능하다', '사람', '도', '회계', '업무', '보다'],
 ['조금', '있다', '헤어지다', '달'],
 ['도대체', '노무현', '빨다', '새끼', '무슨', '생각', '으로', '빨다'],
 ['친구',
  '네',
  '이사',
  '님',
  '친구',
  '한테',
  '자꾸',
  '고가',
  '선물',
  '을',
  '주다',
  '그냥',
  '주다',
  '끄다'],
 ['궁금하다', 'ㅠ'],
 ['퇴', '추다', '시키다', '대선', '때', '우리', '가족', '몰다', '표', '다'],
 ['박근혜',
  '역사',
  '에',
  '비아그라',
  '업무',
  '시간',
  '중',
  '집',
  '에서',
  '보기',
  '성형',
  '시술',
  '마취',
  '해외여행',
  '으로',
  '기록',
  '되다'],
 ['하야', '요구', '무슨', '근거', '로', '법치', '주의', '에', '어긋나다'],
 ['태어나다', '고맙다', '세븐', '틴', '으로', '나오다', '고맙다'],
 ['입',
  '만',
  '열다',
  '민주주의',
  '민주화',
  '를',
  '외치다',
  '안철수',
  '문재인',
  '리',
  '석기',
  '김제동',
  '박원순',
  '이렇다',
  '인간',
  '얼마나',
  '세계',
  '추세',
  '도',
  '못',
  '따라가다',
  '보다',
  '모르다

In [19]:
#테스트 텍스트들도 동일하게 전처리

clean_test_text = []
for text in x_test:
  if type(text) == str:
    clean_test_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_text.append([])
    
clean_test_text

[['생일', '진심', '으로', '축하', '하고', '좋다', '하루', '보내다'],
 ['에', '휴', '눔', '나라', '꼬', '라지', '잘', '돌아가다', '전쟁', '답', '이다'],
 ['어떻다', '하다', '좋다', '알다', 'ㅠㅠ'],
 ['진심', '멋지다', '고생', '하다'],
 ['갑자기', '울컥', '하다'],
 ['남자친구', '때문', '에', '고민', '이에요', 'ㅜ'],
 ['음식',
  '하다',
  '정말',
  '싫다',
  '달콤하다',
  '슈가',
  '보이',
  '님',
  '등장',
  '으로',
  '흥미',
  '만',
  '땅',
  'ㅎㅎ'],
 ['정품', '맞다', '중국산', '짝퉁', '알다', 'ㅋㅋㅋ'],
 ['조선족', '무시무시하다', '니', '네', '나라', '로', '돌아가다'],
 ['꾸준하다', '성실하다', '사람', '좋다', '결실', '맺다', '모습', '너무', '보기', '좋다', 'ㅎㅎ'],
 ['일', '부터는', '부터는', '고삼', '과', '수생', '도', '함께', '하다', '거', '에요'],
 ['공정위', '주머니', '에', '얼마', '들다', '나'],
 ['언플', '임', '런닝맨', '조금', '이라도', '끄다', '강호동', '옹호', '발언', '댓글', '지워지다', '있다'],
 ['근데', '피디', '개', '웃기', '다'],
 ['당황', '하다', '말고', '자리', '에', '가만히', '앉다', '방송', '을', '보다', '기다리다'],
 ['이정로', '선생님', '을', '만나다', '인생', '달라지다'],
 ['중국', '입김', '있다'],
 ['저', '장애인', '되다', '싫다', '한낱', '삼', '성', '폰', '쓰다', '손목', '나르다', '무섭다'],
 ['나', '글', '제목', '처럼', '심각하다', '음치', '야', 'ㅠㅠ'],
 ['내', '일'

In [20]:
lengths = [len(item) for item in clean_train_text]
average_length = sum(lengths) / len(lengths)

print("평균 길이:", average_length)

평균 길이: 8.057539379109956


In [21]:
max(lengths)

112

In [22]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_text)
train_sequences = tokenizer.texts_to_sequences(clean_train_text)
test_sequences = tokenizer.texts_to_sequences(clean_test_text)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 30 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#평가 데이터 
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

In [23]:
train_inputs

array([[  390,    61,     8, ...,     0,     0,     0],
       [ 1033,   737,  7054, ...,     0,     0,     0],
       [   68,  5579,   256, ...,     0,     0,     0],
       ...,
       [  318,   566, 18924, ...,     0,     0,     0],
       [ 1164,     3,  2635, ...,     0,     0,     0],
       [ 1339,     5,  1641, ...,     0,     0,     0]], dtype=int32)

In [24]:
train_inputs.shape

(26156, 30)

In [25]:
train_labels = np.array(y_train)
test_labels = np.array(y_test)

test_labels

array([4, 5, 0, ..., 5, 0, 0], dtype=object)

In [26]:
DEFAULT_PATH  = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/' # 경로지정
DATA_PATH = 'CLEAN_DATA/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)

#전처리 테스트데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)

## 모델링 시작!?

In [27]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm

#전처리 데이터 불러오기
DATA_PATH = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/CLEAN_DATA/'
weights = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA,'rb'))
train_input = pad_sequences(train_input,maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA,'rb'), allow_pickle=True)
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))

In [28]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 50
VALID_SPLIT = 0.1
MAX_LEN = train_inputs.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'], 'embedding_size':256, 'num_filters':30, 'dropout_rate':0.5, 'hidden_dimension':50,'output_dimension':6, 'num_classes': 6}

In [29]:
class CNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(CNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embedding_size'])
        self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid', activation=tf.keras.activations.relu,
                                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3, 4, 5]]
        self.pooling = layers.GlobalMaxPooling1D()
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(units=kargs['hidden_dimension'], activation=tf.keras.activations.relu,
                                 kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
        self.fc2 = layers.Dense(units=kargs['num_classes'], activation=tf.keras.activations.softmax,
                                 kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))

    def call(self, x, training=False):
        x = self.embedding(x)
        x = self.dropout(x, training=training)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=1)
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        x = self.fc2(x)
        return x

In [30]:
train_label = train_label.astype(int)
train_label

array([1, 1, 0, ..., 0, 2, 4])

In [31]:
from tensorflow.keras.models import save_model

model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

#검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)
checkpoint_path = weights + 'weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])
# 모델 저장하기
save_model(model,'/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/model')

/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights -- Folder already exists 

Epoch 1/50
46/46 [==============================] - ETA: 0s - loss: 1.7682 - accuracy: 0.2314
Epoch 1: val_accuracy improved from -inf to 0.35818, saving model to /Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/weights.h5
46/46 [==============================] - 5s 80ms/step - loss: 1.7682 - accuracy: 0.2314 - val_loss: 1.7041 - val_accuracy: 0.3582
Epoch 2/50
46/46 [==============================] - ETA: 0s - loss: 1.5540 - accuracy: 0.3774
Epoch 2: val_accuracy improved from 0.35818 to 0.45833, saving model to /Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/weights.h5
46/46 [==============================] - 3s 73ms/step - loss: 1.5540 - accuracy: 0.3774 - val_loss: 1.3857 - val_accuracy: 0.4583
Epoch 3/50
46/46 [==============================] - ETA: 0s - loss: 1.3244 - accuracy: 0.4630
Epoch 3: val_accuracy improved from 0.45833 to 0.480

INFO:tensorflow:Assets written to: /Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/model/assets


INFO:tensorflow:Assets written to: /Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/model/assets


In [32]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'weights.h5'

test_input = np.load(open(DATA_PATH+INPUT_TEST_DATA,'rb'))
test_input = pad_sequences(test_input,maxlen=test_input.shape[1])
test_label = np.load(open(DATA_PATH + LABEL_TEST_DATA, 'rb'), allow_pickle=True)

# 다중 분류를 위한 One-Hot 인코딩
test_label = test_label.astype(int)

model.load_weights('/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/weights.h5')
model.evaluate(test_input, test_label)

205/205 [==============================] - 0s 2ms/step - loss: 1.2040 - accuracy: 0.5171


[1.2039932012557983, 0.5171253681182861]

In [33]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
tokenizer  = Tokenizer()

DATA_PATH = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/CLEAN_DATA/'
DATA_CONFIGS = 'data_configs.json'
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))
prepro_configs['vocab'] = word_vocab

tokenizer.fit_on_texts(word_vocab)

MAX_LENGTH = 20 #문장최대길이

sentence = input('감성분석할 문장을 입력해 주세요.: ')

#print(sentence)
sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
sentence = okt.morphs(sentence, stem=True) # 토큰화
sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
vector  = tokenizer.texts_to_sequences(sentence)
pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩

#print('변환된 문자 결과값: ',pad_new)
model.load_weights('/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/weights.h5') #모델 불러오기

class_labels = ['불안', '분노', '슬픔', '중립', '행복', '혐오']

predictions = model.predict(pad_new)
max_index = np.argmax(predictions[0])
class_name = class_labels[max_index]
confidence = float(predictions[0, max_index])

print("결과: {}".format(class_name))

감성분석할 문장을 입력해 주세요.:짜증나
1/1 [==============================] - 0s 85ms/step
결과: 불안


In [43]:
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

class SentimentAnalyzer:
    def __init__(self, model_path, weights_path, data_configs_path):
        self.okt = Okt()
        self.tokenizer = Tokenizer()
        self.load_data_configs(data_configs_path)
        self.load_model(model_path, weights_path)
        self.class_labels = ['불안', '분노', '슬픔', '중립', '행복', '혐오']
        self.max_length = 30

    def load_data_configs(self, data_configs_path):
        prepro_configs = json.load(open(data_configs_path, 'r'))
        word_vocab = prepro_configs['vocab']
        self.tokenizer.fit_on_texts(word_vocab)

    def load_model(self, model_path, weights_path):
        self.model = load_model(model_path)
        self.model.load_weights(weights_path)

    def preprocess_sentence(self, sentence):
        sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]', '', sentence)
        stopwords = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']
        sentence = self.okt.morphs(sentence, stem=True)
        sentence = [word for word in sentence if not word in stopwords]
        vector = self.tokenizer.texts_to_sequences(sentence)
        pad_new = pad_sequences(vector, maxlen=self.max_length)
        return pad_new

    def predict_sentiment(self, sentence):
        preprocessed_sentence = self.preprocess_sentence(sentence)
        predictions = self.model.predict(preprocessed_sentence)
        max_index = np.argmax(predictions[0])
        class_name = self.class_labels[max_index]
        confidence = float(predictions[0, max_index])
        return class_name, confidence

In [45]:
from tensorflow.keras.models import load_model

model_path = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/model/'
weights_path = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/weights/weights.h5'
data_configs_path = '/Users/seongjinhyeon/Desktop/codestatesJinHyen/Section4_project/CLEAN_DATA/data_configs.json'

analyzer = SentimentAnalyzer(model_path, weights_path, data_configs_path)

sentence = input('감성분석할 문장을 입력해 주세요.: ')
result, confidence = analyzer.predict_sentiment(sentence)

print("결과: {}".format(result))

감성분석할 문장을 입력해 주세요.: 프로젝트가 끝나서 기분이 좋다.
1/1 [==============================] - 0s 59ms/step
결과: 불안
